# ProtoNN in Tensorflow

This is a simple notebook that illustrates the usage of Tensorflow implementation of ProtoNN. We are using the USPS dataset. Please refer to `fetch_usps.py` for more details on downloading the dataset.

In [1]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT license.

from __future__ import print_function
import sys
import os
import numpy as np
import tensorflow as tf

sys.path.insert(0, '../../')
from edgeml.trainer.protoNNTrainer import ProtoNNTrainer
from edgeml.graph.protoNN import ProtoNN
import edgeml.utils as utils
import helpermethods as helper

# USPS Data

It is assumed that the USPS data has already been downloaded and set up with the help of [fetch_usps.py](fetch_usps.py) and is placed in the `./usps10` subdirectory.

In [2]:
# Load data
DATA_DIR = './usps10'
out = helper.preprocessData(DATA_DIR)
dataDimension = out[0]
numClasses = out[1]
x_train, y_train = out[2], out[3]
x_test, y_test = out[4], out[5]
print("Feature Dimension: ", dataDimension)
print("Num classes: ", numClasses)

Feature Dimension:  256
Num classes:  10


# Model Parameters

Note that ProtoNN is very sensitive to the value of the hyperparameter $\gamma$, here stored in valiable `GAMMA`. If `GAMMA` is set to `None`, median heuristic will be used to estimate a good value of $\gamma$ through the `helper.getGamma()` method. This method also returns the corresponding `W` and `B` matrices which should be used to initialize ProtoNN (as is done here).

In [3]:
PROJECTION_DIM = 60
NUM_PROTOTYPES = 60
REG_W = 0.000005
REG_B = 0.0
REG_Z = 0.00005
SPAR_W = 0.8
SPAR_B = 1.0
SPAR_Z = 1.0
LEARNING_RATE = 0.05
NUM_EPOCHS = 100
BATCH_SIZE = 32
GAMMA = 0.0015

In [4]:
W, B, gamma = helper.getGamma(GAMMA, PROJECTION_DIM, dataDimension,
                       NUM_PROTOTYPES, x_train)

In [5]:
# Setup input and train protoNN
X = tf.placeholder(tf.float32, [None, dataDimension], name='X')
Y = tf.placeholder(tf.float32, [None, numClasses], name='Y')
protoNN = ProtoNN(dataDimension, PROJECTION_DIM,
                  NUM_PROTOTYPES, numClasses,
                  gamma, W=W, B=B)
trainer = ProtoNNTrainer(protoNN, REG_W, REG_B, REG_Z,
                         SPAR_W, SPAR_B, SPAR_Z,
                         LEARNING_RATE, X, Y, lossType='xentropy')
sess = tf.Session()
trainer.train(BATCH_SIZE, NUM_EPOCHS, sess, x_train, x_test, y_train, y_test,
              printStep=600)



Epoch:   0 Batch:   0 Loss: 11.86585 Accuracy: 0.12500
Epoch:   1 Batch:   0 Loss: 1.52476 Accuracy: 0.43750
Epoch:   2 Batch:   0 Loss: 0.83785 Accuracy: 0.84375
Test Loss: 0.66157 Accuracy: 0.85265
Epoch:   3 Batch:   0 Loss: 0.55708 Accuracy: 0.84375
Epoch:   4 Batch:   0 Loss: 0.43675 Accuracy: 0.87500
Epoch:   5 Batch:   0 Loss: 0.36720 Accuracy: 0.93750
Test Loss: 0.52688 Accuracy: 0.88305
Epoch:   6 Batch:   0 Loss: 0.32085 Accuracy: 0.93750
Epoch:   7 Batch:   0 Loss: 0.29802 Accuracy: 0.96875
Epoch:   8 Batch:   0 Loss: 0.29344 Accuracy: 0.96875
Test Loss: 0.49361 Accuracy: 0.89899
Epoch:   9 Batch:   0 Loss: 0.29769 Accuracy: 0.96875
Epoch:  10 Batch:   0 Loss: 0.30366 Accuracy: 0.96875
Epoch:  11 Batch:   0 Loss: 0.31051 Accuracy: 0.96875
Test Loss: 0.50128 Accuracy: 0.90092
Epoch:  12 Batch:   0 Loss: 0.31400 Accuracy: 0.96875
Epoch:  13 Batch:   0 Loss: 0.31334 Accuracy: 0.96875
Epoch:  14 Batch:   0 Loss: 0.30821 Accuracy: 0.96875
Test Loss: 0.51146 Accuracy: 0.89993
Epoc

# Model Evaluation

In [6]:
acc = sess.run(protoNN.accuracy, feed_dict={X: x_test, Y: y_test})
# W, B, Z are tensorflow graph nodes
W, B, Z, _ = protoNN.getModelMatrices()
matrixList = sess.run([W, B, Z])
sparcityList = [SPAR_W, SPAR_B, SPAR_Z]
nnz, size, sparse = helper.getModelSize(matrixList, sparcityList)
print("Final test accuracy", acc)
print("Model size constraint (Bytes): ", size)
print("Number of non-zeros: ", nnz)
nnz, size, sparse = helper.getModelSize(matrixList, sparcityList, expected=False)
print("Actual model size: ", size)
print("Actual non-zeros: ", nnz)

Final test accuracy 0.91679126
Model size constraint (Bytes):  78240
Number of non-zeros:  19560
Actual model size:  78240
Actual non-zeros:  16488
